# 4.0_SynAnalyzer_PlotSynAnalysis_Runx1NIHL.ipynb
Created by: JFranco | 6 AUG 2024 | Env: SynCounting | Last run: 28 JAN 2025

This Python notebook is in the Synapse Analyzer pipeline. The purpose of the notebook is to plot the results of the SynAnalyzer.ijm

In [ ]:
import datetime
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import glob
import seaborn as sns
import math
from scipy import stats
from scipy.stats import ks_2samp

In [ ]:
#                   *** GET TIME OF ANALYSIS START ***
# could turn this into a module?
toa = str(datetime.datetime.today()).split()                                   #Time of analysis
today = toa[0]
now = toa[1]
timestamp = today.replace('-','')+'-'+now.replace(':','')[:6]

In [ ]:
#                *** WHAT TO ANALYZE // WHERE TO GET/STORE **
# Key identifiers
batchID = 'Runx1NIHL_SynAnalyzer'
# Metadata fields to grab from the imaging metadata sheet & transfer to the image summary sheet
mdIFields = ['SlideID','Freq','NoHCRecon']
# Metadata fields to grab from the sample sheet & transfer to the image summary sheet
mdSFields = ['AnimalID','Sex','DOB','Genotype','Treatment','NoiseExposed']

# Surfaces to include in the analysis
surfs = ['PreSyn','PostSyn']
# Define condition (used for plotting) - Variables declared above and pasted here for reference
#   The base level code is setup to test up to two different condition sets
#   If only condition set is used, the variables may appear redundant
condToTest = ['Genotype','NoiseExposed']
condOneVals = ['B5Cre+/-;Runx1Fl+/+','B5Cre-/-;Runx1Fl+/+',]
condTwoVals = ['Exposed','Sham',]
conditions = ['Cre+;Exposed','Cre+;Sham', 'Cre-;Exposed', 'Cre-;Sham']
# Parameters for plotting
pltOrder = conditions
colors = {pltOrder[0]:'rebeccapurple' , pltOrder[1]:'thistle', pltOrder[2]:'darkcyan', pltOrder[3]:'paleturquoise'}
ecdfColors = ['rebeccapurple','thistle','darkcyan','paleturquoise']

# Directories 
#   existing ones 
dirMain = '/Users/joyfranco/Partners HealthCare Dropbox/Joy Franco/JF_Shared/Data/FromCollaborators/Copeland2024/'
dirBA = dirMain+batchID+'/'
dirMD = dirBA+'Metadata/'
dirSA = dirBA+'SAR.Analysis/'

#   ones that need to be made    
dirRes = dirBA+'SAR.Results/'
dirTR = dirRes+timestamp+'/'

# Files to access
fnMDSamp = batchID+'.Metadata.Samples.csv'
fnMDIms = batchID+'.Metadata.Imaging.csv'
fnBM = 'SynAnalyzerBatchMaster.csv'

# Files to make
fnSS = batchID+'.ImageSummarySheet.csv'

In [ ]:
#  *** INITIALIZE RUN SPECIFIC DIRECTORY ETC FOR STORING RESULTS **
# Create directory for storing spreadsheetS and summary plotS for this run
if not os.path.exists(dirRes): os.mkdir(dirRes)
if not os.path.exists(dirTR): os.mkdir(dirTR)

In [ ]:
#  *** LOAD METADATA AND BATCH MASTER FILES ***
dfSamps = pd.read_csv(dirMD+fnMDSamp)
dfSamps.reset_index(inplace=True)
dfSamps.drop('index', axis=1, inplace=True)
dfIms = pd.read_csv(dirMD+fnMDIms)
dfIms.reset_index(inplace=True)
dfIms.drop('index', axis=1, inplace=True)
dfBM = pd.read_csv(dirSA+fnBM)
dfBM.reset_index(inplace=True)

In [ ]:
#  *** COMPILE BATCH MASTER DATA ***
# Add metadata for each image to the batch master and calculate image stats
imList = dfIms['ImageName']
for im in imList:

    # Get the index of the image in Ims MD df
    inIm = dfIms.index[dfIms['ImageName']==im].tolist()[0]
    # Get the index of the image in Batch Master
    inBM = dfBM.index[dfBM['ImageName']==(im+'.czi')].tolist()[0]
    # Check that the image has reconstructed hair cells
    noHCRecon = dfIms.loc[inIm]['NoHCRecon']
    if not (math.isnan(noHCRecon)):
        # For images that have valid hair cell reconstructed information --->
        # Get other key information from imaging MD sheet
        samp = dfIms.loc[inIm]['SlideID']
        freq = dfIms.loc[inIm]['Freq']
        # Get information from sample MD sheet
        inSamp = dfSamps.index[dfSamps['SampleID']==samp].tolist()[0]
        anID = dfSamps.loc[inSamp]['AnimalID']
        geno = dfSamps.loc[inSamp]['Genotype']
        treat = dfSamps.loc[inSamp]['Treatment']
        neStatus = dfSamps.loc[inSamp]['NoiseExposed']
        # Get information on the number of synapses
        pSyn = int(dfBM.loc[inBM]['PreSyn_nPillar'])
        mSyn = int(dfBM.loc[inBM]['PreSyn_nModiolar'])
        nSyn = int(dfBM.loc[inBM]['PreSyn_nSynapses'])

        # Add all relevant information to this entry of the Batch Master
        dfBM.at[inBM, 'AnimalID'] = anID
        dfBM.at[inBM, 'Genotype'] = geno
        dfBM.at[inBM, 'Treatment'] = treat
        dfBM.at[inBM, 'NoiseExposed'] = neStatus
        dfBM.at[inBM, 'Freq'] = freq
        dfBM.at[inBM, 'NoHC'] = noHCRecon

        dfBM.at[inBM,'ModProp']= mSyn/(pSyn+mSyn)
        dfBM.at[inBM,'SynPerHC']= nSyn/noHCRecon

In [ ]:
#  *** COMPILE XYZ DATA ***
# Initialize a data frame for storing information for every sample 
dfISS = pd.DataFrame()
# Initialize a data frame for storying all XYZ data
dfXYZAll = pd.DataFrame()
# Iterate through the available XYZ sheets from SynAnalyzer to compile the results
imList = dfIms['ImageName']
for im in imList:
    # Get the index of the image in Ims MD df
    inIm = dfIms.index[dfIms['ImageName']==im].tolist()[0]
    # Check that the image has reconstructed hair cells
    noHCRecon = dfIms.loc[inIm]['NoHCRecon']
    if not (math.isnan(noHCRecon)):
        # For images that have valid hair cell reconstructed information --->
        # Get other key information from imaging MD sheet
        samp = dfIms.loc[inIm]['SlideID']
        freq = dfIms.loc[inIm]['Freq']
        noHC = dfIms.loc[inIm]['NoHCRecon']
        # Geet information from sample MD sheet
        inSamp = dfSamps.index[dfSamps['SampleID']==samp].tolist()[0]
        animalID = dfSamps.loc[inSamp]['AnimalID']
        sex = dfSamps.loc[inSamp]['Sex']
        dob = dfSamps.loc[inSamp]['DOB']
        geno = dfSamps.loc[inSamp]['Genotype']
        treat = dfSamps.loc[inSamp]['Treatment']
        neStatus = dfSamps.loc[inSamp]['NoiseExposed']
        
        # Define condition (used for plotting) - Variables declared above and pasted here for reference
        # condToTest = ['Genotype','NoiseExposed']
        # condOneVals = ['B5Cre+/-;Runx1Fl+/+','B5Cre-/-;Runx1Fl+/+',]
        # condTwoVals = ['Exposed','Sham',]
        # conditions = ['Cre+ Exposed','Cre+ Sham', 'Cre- Exposed', 'Cre- Sham']
        if ((geno == condOneVals[0]) & (neStatus == condTwoVals[0])):
            # Condition = Cre+ Exposed
            cond = conditions[0]
            print(cond)
        elif ((geno == condOneVals[0]) & (neStatus == condTwoVals[1])):
            # Condition = Cre+ Sham 
            cond = conditions[1]
        elif ((geno == condOneVals[1]) & (neStatus == condTwoVals[0])):
            # Condition = Cre- Exposed 
            cond = conditions[2]
        elif ((geno == condOneVals[1]) & (neStatus == condTwoVals[1])):
            # Condition = Cre- Sham 
            cond = conditions[3]

        # Setup the df for this image to add to add to dfISS
        df = pd.DataFrame({'ImageName': im, 
                           'SampleID':samp, 
                           'Frequency':[freq],
                           'NoHCRecon':[noHC],
                           'AnimalID':animalID,
                           'Sex':sex,
                           'DOB':dob,
                           'Genotype':geno,
                           'Treatment':treat,
                           'NoiseExposed':neStatus,
                           'Condition':cond})   

        # Iterate through surfs for this image to capture 
        i = 0
        for surf in surfs:
            # Setup the filename to open
            fnXYZ = im+'.XYZ.'+surf+'.csv'
            if os.path.exists(dirSA+fnXYZ):
                # Load the XYZ data
                dfXYZ = pd.read_csv(dirSA+fnXYZ)
                # Add important md to each row
                dfXYZ['ImageName']=im
                dfXYZ['SampleID']=samp
                dfXYZ['Genotype']=geno
                dfXYZ['NoiseExposed']=neStatus
                dfXYZ['Frequency']=freq
                dfXYZ['Surf']=surf
                dfXYZ['Condition']=cond

                # Correct for the pillar-modiolar status bug
                if 'Pillar-ModiolarStatus' in dfXYZ.columns:
                    dfXYZ['PillarModiolarStatus']=dfXYZ['Pillar-ModiolarStatus']
                    xyzs = dfXYZ['ID']
                    for id in xyzs:
                        inID = dfXYZ.index[dfXYZ['ID']==id].tolist()[0]
                        pmStatus = dfXYZ.loc[inID]['PillarModiolarStatus']
                        if(pmStatus=='0'):
                            dfXYZ.at[inID, 'PillarModiolarStatus']='Pillar'
                    
                # Count the number of surfaces and categories
                nAll = len(dfXYZ)
                nSyns = len(dfXYZ[dfXYZ['SynapseStatus']==('Synapse' or 'Doublet')])
                nOrph = len(dfXYZ[dfXYZ['SynapseStatus']==('Orphan')])
                nPill = len(dfXYZ[(dfXYZ['PillarModiolarStatus']==('Pillar')) & (dfXYZ['SynapseStatus']==('Synapse' or 'Doublet'))])
                nMod = len(dfXYZ[(dfXYZ['PillarModiolarStatus']==('Modiolar')) & (dfXYZ['SynapseStatus']==('Synapse' or 'Doublet'))])
                uVol = dfXYZ['Volume_um3'].mean()
                uCh1 = dfXYZ['uIntCh_1'].mean()
                uCh2 = dfXYZ['uIntCh_2'].mean()
                uCh3 = dfXYZ['uIntCh_3'].mean()
                uCh4 = dfXYZ['uIntCh_4'].mean()
                maxCh2 = dfXYZ['uIntCh_4'].max()
                
                # Add summary data to image df
                df[surf+'_Total']=nAll
                df[surf+'_nSyns']=nSyns
                df[surf+'_nOrph']=nOrph
                df[surf+'_nPill']=nPill
                df[surf+'_nMod']=nMod
                df['Avg_'+surf+'_SynPerHC']=nSyns/noHC
                df[surf+'_PropPill']=nPill/nSyns
                df[surf+'_PropMod']=nMod/nSyns
                df[surf+'_uVol_um3']= uVol
                df[surf+'_uIntCh1']= uCh1
                df[surf+'_uIntCh2']= uCh2
                df[surf+'_uIntCh3']= uCh3
                df[surf+'_uIntCh4']= uCh4
                df[surf+'_maxIntCh2']= maxCh2

                # Calculate the mean normalized vals for each surface
                dfXYZ['uNormVol'] = dfXYZ['Volume_um3']/uVol
                dfXYZ['uNormIntCh1'] = dfXYZ['uIntCh_1']/uCh1
                dfXYZ['uNormIntCh2'] = dfXYZ['uIntCh_2']/uCh2
                dfXYZ['uNormIntCh3'] = dfXYZ['uIntCh_3']/uCh3
                dfXYZ['uNormIntCh4'] = dfXYZ['uIntCh_4']/uCh4
                dfXYZ['maxNormIntCh2'] = dfXYZ['uIntCh_4']/maxCh2

                # Add it to the main df
                dfXYZAll = pd.concat([dfXYZAll,dfXYZ])
                i = i+1
            if(i==2):
                # Add information to the image summary sheet df
                dfISS = pd.concat([dfISS,df])

In [ ]:
# Save the updated sheets
dfXYZAll.to_csv(dirTR+batchID+'.XYZSummary.csv') 
dfISS.to_csv(dirTR+batchID+'.ImageSummary.csv') 

### GENERATE PLOTS

In [ ]:
# *** SET PLOT SPECIFIC PARAMETERS ***
plot = 'PostSyn_AvgSynPerHC'
plottype = 'BoxSwarm'
region = 32
title = 'Synapses per Hair Cell ('+str(region)+' kHz)\n(based on Homer1 reconstructions)'
hue = 'Condition'
x = 'Condition'
y = 'Avg_PostSyn_SynPerHC'
ymin = 0
ymax = 20
xlab = 'Condition'
ylab = 'Synapses per HC'

# *** MAKE PLOT ***
fig= plt.figure(figsize=(4,3))
plt.subplots_adjust(top = 0.8, bottom=0.04, hspace=.3, wspace=.2)
fig.suptitle(title)
ax1 = sns.swarmplot(data=dfISS[(dfISS['Frequency']==region)],x=x,y=y, hue=hue,palette=colors,order=pltOrder,legend = False, dodge = False)
sns.boxplot(data=dfISS[(dfISS['Frequency']==region)],x=x,y=y, hue=hue,palette=colors,order=pltOrder,boxprops={'alpha': 0.1}, ax=ax1, dodge = False,linewidth=.5)
ax1.set_ylim((ymin, ymax))
ax1.set(xlabel=xlab)
ax1.set(ylabel=ylab)
ax1.get_legend().remove()

# *** SAVE THE FIGURE ***
plt.savefig(dirTR+batchID+'.'+plot+'.'+plottype+'.'+str(region)+'.png',  bbox_inches='tight', dpi = 300,format = 'png')

In [ ]:
# *** SET PLOT SPECIFIC PARAMETERS ***
plot = 'PreSyn_AvgSynPerHC'
plottype = 'BoxSwarm'
region = 32
title = 'Synapses per Hair Cell ('+str(region)+' kHz)\n(based on CtBP2 reconstructions)'
hue = 'Condition'
x = 'Condition'
y = 'Avg_PreSyn_SynPerHC'
ymin = 0
ymax = 25
xlab = 'Condition'
ylab = 'Synapses per HC'

# *** MAKE PLOT ***
fig= plt.figure(figsize=(4,3))
plt.subplots_adjust(top = 0.8, bottom=0.04, hspace=.3, wspace=.2)
fig.suptitle(title)
ax1 = sns.swarmplot(data=dfISS[(dfISS['Frequency']==region)],x=x,y=y, hue=hue,palette=colors,order=pltOrder,legend = False, dodge = False)
sns.boxplot(data=dfISS[(dfISS['Frequency']==region)],x=x,y=y, hue=hue,palette=colors,order=pltOrder,boxprops={'alpha': 0.1}, ax=ax1, dodge = False,linewidth=.5)
ax1.set_ylim((ymin, ymax))
ax1.set(xlabel=xlab)
ax1.set(ylabel=ylab)
ax1.get_legend().remove()

# *** SAVE THE FIGURE ***
plt.savefig(dirTR+batchID+'.'+plot+'.'+plottype+'.'+str(region)+'.png',  bbox_inches='tight', dpi = 300,format = 'png')

In [ ]:
# *** SET PLOT SPECIFIC PARAMETERS ***
plot = 'PostSyn_PropModSyns'
plottype = 'BoxSwarm'
region = 32
title = 'Proportion of Modiolar Synapses ('+str(region)+' kHz)\n(based on Homer1 reconstructions)'
hue = 'Condition'
x = 'Condition'
y = 'PostSyn_PropMod'
ymin = 0
ymax = 1
xlab = 'Condition'
ylab = 'Modiolar Synapses\n(Proportion of all Synapses)'

# *** MAKE PLOT ***
fig= plt.figure(figsize=(4,3))
plt.subplots_adjust(top = 0.8, bottom=0.04, hspace=.3, wspace=.2)
fig.suptitle(title)
ax1 = sns.swarmplot(data=dfISS[(dfISS['Frequency']==region)],x=x,y=y, hue=hue,palette=colors,order=pltOrder,legend = False, dodge = False)
sns.boxplot(data=dfISS[(dfISS['Frequency']==region)],x=x,y=y, hue=hue,palette=colors,order=pltOrder,boxprops={'alpha': 0.1}, ax=ax1, dodge = False,linewidth=.5)
ax1.set_ylim((ymin, ymax))
ax1.set(xlabel=xlab)
ax1.set(ylabel=ylab)
ax1.get_legend().remove()

# *** SAVE THE FIGURE ***
plt.savefig(dirTR+batchID+'.'+plot+'.'+plottype+'.'+str(region)+'.png',  bbox_inches='tight', dpi = 300,format = 'png')

In [ ]:
# *** SET PLOT SPECIFIC PARAMETERS ***
plot = 'PreSyn_PropModSyns'
plottype = 'BoxSwarm'
region = 32
title = 'Proportion of Modiolar Synapses ('+str(region)+' kHz)\n(based on CtBP2 reconstructions)'
hue = 'Condition'
x = 'Condition'
y = 'PreSyn_PropMod'
ymin = 0
ymax = 1
xlab = 'Condition'
ylab = 'Modiolar Synapses\n(Proportion of all Synapses)'

# *** MAKE PLOT ***
fig= plt.figure(figsize=(4,3))
plt.subplots_adjust(top = 0.8, bottom=0.04, hspace=.3, wspace=.2)
fig.suptitle(title)
ax1 = sns.swarmplot(data=dfISS[(dfISS['Frequency']==region)],x=x,y=y, hue=hue,palette=colors,order=pltOrder,legend = False, dodge = False)
sns.boxplot(data=dfISS[(dfISS['Frequency']==region)],x=x,y=y, hue=hue,palette=colors,order=pltOrder,boxprops={'alpha': 0.1}, ax=ax1, dodge = False,linewidth=.5)
ax1.set_ylim((ymin, ymax))
ax1.set(xlabel=xlab)
ax1.set(ylabel=ylab)
ax1.get_legend().remove()

# *** SAVE THE FIGURE ***
plt.savefig(dirTR+batchID+'.'+plot+'.'+plottype+'.'+str(region)+'.png',  bbox_inches='tight', dpi = 300,format = 'png')

In [ ]:
# *** SET PLOT SPECIFIC PARAMETERS ***
colors = {'Pillar': 'darkturquoise', 'Modiolar': 'darkorange'}
group = 'Cre+;Sham'
plottype = 'ECDF'
region = 32

surfType = 'PreSyn'
hue = 'Pillar-ModiolarStatus'
x = 'Volume_um3'
xlab = 'Volume (um3)'
ylab = 'Proportion of Surfaces'
title = surfType+' Surface Volume '+ group
plot = surfType+'RawVolume_'+group

# *** MAKE PLOT ***
plt.figure(figsize=(5, 3))
ax = sns.ecdfplot(data=dfXYZAll[(dfXYZAll['Frequency']==region)&(dfXYZAll['Surf']==surfType)&
                  (dfXYZAll['Condition']==group)],x=x, hue=hue,palette=colors)
ax.set_title(title)
ax.set(xlabel=xlab)
ax.set(ylabel=ylab)

# *** SAVE THE FIGURE ***
plt.savefig(dirTR+batchID+'.'+plot+'.'+plottype+'.'+str(region)+'.png',  bbox_inches='tight', dpi = 300,format = 'png')